In [5]:
#Datasets
###ML-100K


!wget "http://files.grouplens.org/datasets/movielens/ml-100k.zip"
!unzip ml-100k.zip
!ls



--2020-10-19 21:52:30--  http://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip’

ml-100k.zip         100%[===================>]   4.70M  3.51MB/s    in 1.3s    

2020-10-19 21:52:32 (3.51 MB/s) - ‘ml-100k.zip’ saved [4924029/4924029]

Archive:  ml-100k.zip
   creating: ml-100k/
  inflating: ml-100k/allbut.pl       
  inflating: ml-100k/mku.sh          
  inflating: ml-100k/README          
  inflating: ml-100k/u.data          
  inflating: ml-100k/u.genre         
  inflating: ml-100k/u.info          
  inflating: ml-100k/u.item          
  inflating: ml-100k/u.occupation    
  inflating: ml-100k/u.user          
  inflating: ml-100k/u1.base         
  inflating: ml-100k/u1.test         
  inflating: ml-100k/u2.base

In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [6]:
training_set = pd.read_csv('ml-100k/u1.base', delimiter = '\t')
training_set = np.array(training_set, dtype = 'int')
test_set = pd.read_csv('ml-100k/u1.test', delimiter = '\t')
test_set = np.array(test_set, dtype = 'int')

In [7]:
"""## Getting the number of users and movies"""

nb_users = int(max(max(training_set[:, 0], ), max(test_set[:, 0])))
nb_movies = int(max(max(training_set[:, 1], ), max(test_set[:, 1])))

In [8]:
"""## Converting the data into an array with users in lines and movies in columns"""

def convert(data):
  new_data = []
  for id_users in range(1, nb_users + 1):
    id_movies = data[:, 1] [data[:, 0] == id_users]
    id_ratings = data[:, 2] [data[:, 0] == id_users]
    ratings = np.zeros(nb_movies)
    ratings[id_movies - 1] = id_ratings
    new_data.append(list(ratings))
  return new_data
training_set = convert(training_set)
test_set = convert(test_set)

In [9]:
"""## Converting the data into Torch tensors"""

training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

In [11]:
"""## Creating the architecture of the Stacked AE"""

class SAE(nn.Module):
    def __init__(self, ):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_movies, 30)
        self.fc2 = nn.Linear(30, 10)
        self.fc3 = nn.Linear(10, 30)
        self.fc4 = nn.Linear(30, nb_movies)
        self.activation = nn.Sigmoid()
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5)

In [12]:
"""## Training the AE"""

nb_epoch = 250
for epoch in range(1, nb_epoch + 1):
  train_loss = 0
  s = 0.
  for id_user in range(nb_users):
    input = Variable(training_set[id_user]).unsqueeze(0)
    target = input.clone()
    if torch.sum(target.data > 0) > 0:
      output = sae(input)
      target.require_grad = False
      output[target == 0] = 0
      loss = criterion(output, target)
      mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
      loss.backward()
      train_loss += np.sqrt(loss.data*mean_corrector)
      s += 1.
      optimizer.step()
  print('epoch: '+str(epoch)+'loss: '+ str(train_loss/s))

epoch: 1loss: tensor(1.6384)
epoch: 2loss: tensor(1.0779)
epoch: 3loss: tensor(1.0497)
epoch: 4loss: tensor(1.0401)
epoch: 5loss: tensor(1.0362)
epoch: 6loss: tensor(1.0336)
epoch: 7loss: tensor(1.0325)
epoch: 8loss: tensor(1.0311)
epoch: 9loss: tensor(1.0305)
epoch: 10loss: tensor(1.0301)
epoch: 11loss: tensor(1.0296)
epoch: 12loss: tensor(1.0288)
epoch: 13loss: tensor(1.0287)
epoch: 14loss: tensor(1.0281)
epoch: 15loss: tensor(1.0248)
epoch: 16loss: tensor(1.0220)
epoch: 17loss: tensor(1.0226)
epoch: 18loss: tensor(1.0183)
epoch: 19loss: tensor(1.0181)
epoch: 20loss: tensor(1.0133)
epoch: 21loss: tensor(1.0138)
epoch: 22loss: tensor(1.0088)
epoch: 23loss: tensor(1.0100)
epoch: 24loss: tensor(1.0068)
epoch: 25loss: tensor(1.0071)
epoch: 26loss: tensor(1.0026)
epoch: 27loss: tensor(0.9996)
epoch: 28loss: tensor(0.9970)
epoch: 29loss: tensor(0.9967)
epoch: 30loss: tensor(0.9946)
epoch: 31loss: tensor(0.9914)
epoch: 32loss: tensor(0.9886)
epoch: 33loss: tensor(0.9848)
epoch: 34loss: tens

In [13]:
"""## Testing the SAE"""

test_loss = 0
s = 0.
for id_user in range(nb_users):
  input = Variable(training_set[id_user]).unsqueeze(0)
  target = Variable(test_set[id_user]).unsqueeze(0)
  if torch.sum(target.data > 0) > 0:
    output = sae(input)
    target.require_grad = False
    output[target == 0] = 0
    loss = criterion(output, target)
    mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
    test_loss += np.sqrt(loss.data*mean_corrector)
    s += 1.
print('test loss: '+str(test_loss/s))

test loss: tensor(0.9472)


In [14]:
input = Variable(training_set[5]).unsqueeze(0)

In [22]:
output[[0]]

tensor([[3.7325, 3.2545, 2.7263,  ..., 1.9190, 3.1764, 2.9804]],
       grad_fn=<IndexBackward>)